### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable



### Load Dataset

In [2]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
 
test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

### Make Dataset Iterable

In [3]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### Model

In [20]:
class MYGRU(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(MYGRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
               
    def forward(self, x, hidden):
        
        x = x.view(-1, x.size(1))
        
        gate_x = self.x2h(x).squeeze() 
        gate_h = self.h2h(hidden).squeeze()
        
        input_reset, input_z, input_h = gate_x.chunk(3, 1)
        hidden_reset, hidden_z, hidden_h = gate_h.chunk(3, 1)
        
        
        resetgate = torch.sigmoid(input_reset + hidden_reset)
        zgate = torch.sigmoid(input_z + hidden_z)
        hgate = torch.tanh(input_h + (resetgate * hidden_h))
        
        output = hgate*zgate + hidden-zgate
        
        
        return output

class MYLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, bias=True):
        super(MYLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        
    def forward(self, x, hidden):
        hin, cin = hidden
        x = x.view(-1, x.size(1))
        
        gates = (self.x2h(x) + self.h2h(hin)).squeeze()
        
        inputgate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        
        inputgate = torch.sigmoid(inputgate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)
        

        cout = torch.mul(cin, forgetgate) +  torch.mul(inputgate, cellgate)        

        hout = torch.mul(outgate, torch.tanh(cout))
        
        return (hout, cout)



    
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
         
        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = MYLSTM(input_dim, hidden_dim, layer_dim)
        
        #[b_ig | b_fg | b_gg | b_og]
        #print(self.lstm._all_weights) [['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0'], 
        #['weight_ih_l1', 'weight_hh_l1', 'bias_ih_l1', 'bias_hh_l1'], 
        #['weight_ih_l2', 'weight_hh_l2', 'bias_ih_l2', 'bias_hh_l2']]
   
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        
        #print(x.shape,"x.shape")100, 28, 28
        if torch.cuda.is_available():
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
        else:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
         
        # Initialize cell state
        if torch.cuda.is_available():
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
        else:
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        
        outs = []
        
        cn = c0[0,:,:]
        hn = h0[0,:,:]

        for seq in range(x.size(1)):
            hn, cn = self.lstm(x[:,seq,:], (hn,cn)) 
            outs.append(hn)
            
    

        out = self.fc(outs[-1].squeeze())
        
        out = self.fc(out) 
        return out

    
    
class MYGRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(MYGRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
        self.gru_cell = MYGRU(input_dim, hidden_dim, layer_dim)   
        self.fc = nn.Linear(hidden_dim, output_dim)
     
    def forward(self, x):
        
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
         
       
        outs = []
        hn = h0[0,:,:]
        
        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:], hn) 
            outs.append(hn)
            
        out = outs[-1].squeeze()        
        out = self.fc(out) 
        
        return out


### INSTANTIATE MODEL CLASS

In [21]:
input_dim = 28
hidden_dim = 100
layer_dim = 3  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 10
 
model_gru = MYGRUModel(input_dim, hidden_dim, layer_dim, output_dim)
model_lstm = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

 
#######################
#  USE GPU FOR MODEL  #
#######################
 
if torch.cuda.is_available():
    model_gru.cuda()
    model_lstm.cuda()

### Train (LSTM)

In [22]:

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1 
optimizer = torch.optim.SGD(model_lstm.parameters(), lr=learning_rate)  
 
# Number of steps to unroll
seq_dim = 28 
 
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.view(-1, seq_dim, input_dim).cuda()
            labels = labels.cuda()
        else:
            images = images.view(-1, seq_dim, input_dim)

             
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model_lstm(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = images.view(-1, seq_dim, input_dim).cuda()

                 
                # Forward pass only to get logits/output
                outputs = model_lstm(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.0342352390289307. Accuracy: 32.43000030517578
Iteration: 1000. Loss: 0.6216239333152771. Accuracy: 80.66999816894531
Iteration: 1500. Loss: 0.37882065773010254. Accuracy: 90.08000183105469
Iteration: 2000. Loss: 0.10445079207420349. Accuracy: 94.36000061035156
Iteration: 2500. Loss: 0.30027344822883606. Accuracy: 94.5199966430664
Iteration: 3000. Loss: 0.04714415594935417. Accuracy: 96.36000061035156
Iteration: 3500. Loss: 0.073410265147686. Accuracy: 96.80000305175781
Iteration: 4000. Loss: 0.05749019235372543. Accuracy: 96.97000122070312
Iteration: 4500. Loss: 0.10458479821681976. Accuracy: 97.22000122070312
Iteration: 5000. Loss: 0.07756815105676651. Accuracy: 97.5199966430664
Iteration: 5500. Loss: 0.11008927971124649. Accuracy: 97.76000213623047
Iteration: 6000. Loss: 0.09541978687047958. Accuracy: 97.04000091552734


### Train (GRU)

In [23]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1 
optimizer = torch.optim.SGD(model_gru.parameters(), lr=learning_rate)  
 
# Number of steps to unroll
seq_dim = 28 
 
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.view(-1, seq_dim, input_dim).cuda()
            labels = labels.cuda()
        else:
            images = images.view(-1, seq_dim, input_dim)

             
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model_gru(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = images.view(-1, seq_dim, input_dim).cuda()

                 
                # Forward pass only to get logits/output
                outputs = model_gru(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.4096072912216187. Accuracy: 51.189998626708984
Iteration: 1000. Loss: 0.6946542859077454. Accuracy: 72.2300033569336
Iteration: 1500. Loss: 0.5480771064758301. Accuracy: 80.69999694824219
Iteration: 2000. Loss: 0.44128066301345825. Accuracy: 87.4800033569336
Iteration: 2500. Loss: 0.35506847500801086. Accuracy: 88.44999694824219
Iteration: 3000. Loss: 0.4087640047073364. Accuracy: 90.4000015258789
Iteration: 3500. Loss: 0.26842477917671204. Accuracy: 91.95999908447266
Iteration: 4000. Loss: 0.16903333365917206. Accuracy: 91.54000091552734
Iteration: 4500. Loss: 0.25129103660583496. Accuracy: 93.7300033569336
Iteration: 5000. Loss: 0.10783208906650543. Accuracy: 94.11000061035156
Iteration: 5500. Loss: 0.1530032455921173. Accuracy: 93.7300033569336
Iteration: 6000. Loss: 0.11600309610366821. Accuracy: 94.66000366210938
